In [ ]:
import pymysql
import re
import nltk
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# ✅ NLTK 캐싱 적용 (최초 1회 다운로드 후 캐싱)
nltk_data_path = "/home/user/nltk_data"
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)
nltk.data.path.append(nltk_data_path)

if not os.path.exists(os.path.join(nltk_data_path, "corpora/stopwords")):
    nltk.download("stopwords", download_dir=nltk_data_path)

# ✅ 불용어 리스트 (NLTK + sklearn + 커스텀)
custom_stop_words = set(stopwords.words('english')) | ENGLISH_STOP_WORDS

# ✅ 중요 단어 리스트 (삭제 금지)
custom_important_words = {
    # 주요 코인 및 관련 용어 (우리 프로젝트에서 다루는 10개 코인 포함)
    'bitcoin', 'crypto', 'blockchain', 'btc', 'eth', 'ethereum', 'xrp', 'doge', 'sol', 'mtl', 
    'neo', 'waves', 'ripple', 'sensex', 'nft', 'solana', 'xrpl', 'ada', 'bnb', 'dot', 'ltc', 
    'link', 'qtum', 'xem', 'etc', 'snt',

    # 금융 및 기관 관련
    'grayscale', 'blackrock', 'fidelity', 'ark', 'arkinvest', 'vanguard', 'galaxy', 
    'coinshares', 'bitwise', 'paypal', 'square', 'tesla',

    # 주요 인물
    'satoshi', 'cz', 'changpeng', 'sam', 'bankman', 'fried', 'justin', 'sun', 'cathie', 
    'wood', 'elon', 'musk', 'michael', 'saylor', 'dorsey',

    # 금융 지표 및 트레이딩 용어
    'microstrategy', 'hashrate', 'onchain', 'bull', 'bear', 'whale', 'deflation', 
    'inflation', 'supply', 'halving', 'etf', 'fund', 'price', 'traded', 'investment', 
    'liquidity', 'profit', 'profits', 'return', 'returns', 'margin', 'yield',

    # 암호화폐 거래소 및 기업
    'binance', 'coinbase', 'coinglass', 'kuande', 'opensea',

    # 기타 기업 및 테크 용어
    'apple', 'microsoft', 'wikileaks', 'strategy', 'meme', 'memecoin',

    # 추가된 인물 및 기관
    'milei', 'paul', 'javier', 'ceo', 'cftc', 'sec', 'securities', 'contracts', 'fintech', 
    'exchange',

    # 추가된 뉴스 키워드
    'barstool', 'business', 'bse', 'cboe', 'chan', 'coin', 'coinpedia', 'commission', 
    'consensus', 'dave', 'defi', 'divatives', 'dog', 'dr', 'etc', 'futures', 'holdings', 
    'hu', 'intelligence', 'investment', 'libra', 'lps', 'macos', 'network', 'nifty', 
    'phrase', 'pi', 'portnoy', 'post', 'qtum', 'seed', 'share', 'snt', 'tl', 'us', 
    'wizardquant'
}

# ✅ MySQL 연결 설정
DB_CONFIG = {
    "host": "localhost",
    "user": "*****_user",
    "password": "*********",
    "database": "*****_db",
    "cursorclass": pymysql.cursors.DictCursor
}

# ✅ 텍스트 전처리 함수 (불용어 제거, 중요 단어 유지)
def clean_text(text):
    if not text:
        return ""
    
    text = text.lower()  # 소문자 변환
    text = re.sub(r'\b비트 코인\b', '비트코인', text)  # 띄어쓰기 수정
    text = re.sub(r'\b이더 리움\b', '이더리움', text)

    # ✅ 특수 문자 처리 (달러 기호 등 유지, 쉼표 제거)
    text = re.sub(r'[\$₿]', '달러', text)  # "$" → "달러" 변환
    text = re.sub(r'[^\w\s%]', '', text)  # 퍼센트(%) 기호 유지하고 나머지 특수문자 제거
    text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
    
    words = text.split()
    cleaned_words = [word for word in words if word in custom_important_words or word not in custom_stop_words]
    return ' '.join(cleaned_words)

# ✅ 전처리 실행
def preprocess_news():
    conn = pymysql.connect(**DB_CONFIG)
    cursor = conn.cursor()
    
    # ✅ news 테이블에서 전처리할 데이터 가져오기
    cursor.execute("SELECT news_id, title_en, content_en FROM news;")
    rows = cursor.fetchall()

    for row in rows:
        news_id = row["news_id"]
        title_en = row["title_en"]
        content_en = row["content_en"]

        # ✅ 전처리 수행
        processed_title = clean_text(title_en)
        processed_content = clean_text(content_en)

        # ✅ 변환된 내용 로그로 출력 (확인용)
        print(f"📰 원본 제목: {title_en} → 변환 후: {processed_title}")
        print(f"📄 원본 본문: {content_en[:50]}... → 변환 후: {processed_content[:50]}...\n")

        # ✅ 전처리된 데이터 news 테이블에 UPDATE
        update_query = """
            UPDATE news
            SET title_preprocessed = %s, content_preprocessed = %s
            WHERE news_id = %s;
        """
        cursor.execute(update_query, (processed_title, processed_content, news_id))

    conn.commit()
    cursor.close()
    conn.close()

    print(f"✅ 총 {len(rows)}개 뉴스 데이터 전처리 완료!")

# ✅ 실행 (직접 실행 시)
if __name__ == "__main__":
    preprocess_news()
